<a href="https://colab.research.google.com/github/CAU-CAPSTONE-2-Friendly-Keyboard/Models/blob/main/%EB%B9%84%EC%86%8D%EC%96%B4%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 설치

In [ ]:
!pip install transformers==4.1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 32.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 57.6 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=7b5c8434a9cd95e29ae20e092fc7dc2a39f825c4fe3523440a40603584b132c9
  Stored in directory: /root/.cache/pip/wheels/12/1c/3d/46cf06718d63a32ff798a89594b61e7f345ab6b36d909ce033
Successfully built sacremoses
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.


## Importing

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModelForSequenceClassification, AutoTokenizer

## pretrained Model 불러오기

In [ ]:


model = AutoModelForSequenceClassification.from_pretrained('JminJ/kcElectra_base_Bad_Sentence_Classifier')
tokenizer = AutoTokenizer.from_pretrained('JminJ/kcElectra_base_Bad_Sentence_Classifier')


Downloading:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/498M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/396k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/693 [00:00<?, ?B/s]

Attention을 이용한 Masking 연결을 위한 기존 모델 attention 구조 및 연산 확인

In [ ]:
layernorm = nn.LayerNorm(768)

In [ ]:
layers = model.electra.encoder.layer
tmp_query = layers[0].attention.self.query.weight
for layer in layers[1:]:
  tmp_query = torch.add(tmp_query,layer.attention.self.query.weight)
  print(tmp_query)
query_result = layernorm(tmp_query)

print(query_result.shape)
print(query_result)

tensor([[-0.0313,  0.0637,  0.0118,  ...,  0.0840,  0.0239, -0.0226],
        [ 0.1325, -0.0024, -0.0972,  ...,  0.0238,  0.0708, -0.1140],
        [ 0.0134, -0.0985,  0.0316,  ...,  0.0356,  0.1798, -0.0309],
        ...,
        [ 0.1682,  0.0282, -0.0483,  ..., -0.0115, -0.0993,  0.0424],
        [ 0.0239, -0.0051,  0.1035,  ..., -0.0839, -0.0149, -0.1470],
        [ 0.0264, -0.0629,  0.1732,  ...,  0.1792,  0.0791, -0.0692]],
       grad_fn=<AddBackward0>)
tensor([[-0.0224,  0.0687,  0.0400,  ...,  0.1129,  0.0041, -0.0162],
        [ 0.1064, -0.0062, -0.1138,  ...,  0.0996,  0.1383, -0.1605],
        [ 0.1204, -0.1723,  0.0691,  ...,  0.0011,  0.1476, -0.0410],
        ...,
        [ 0.1530,  0.0350, -0.1209,  ..., -0.0819,  0.0037, -0.0195],
        [ 0.0744,  0.0670,  0.0794,  ..., -0.0786, -0.0490, -0.2046],
        [ 0.0226, -0.0184,  0.0607,  ...,  0.1737,  0.0227,  0.0250]],
       grad_fn=<AddBackward0>)
tensor([[-0.0212,  0.1165,  0.2464,  ...,  0.1626,  0.1504,  0.0431],


## 모델

In [ ]:
class FriendlyModel(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self,x):
    return x
